In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import pandas as pd
import os 
import inspect

# Put all datasets in 'path-to-repo/data' directory please
path = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))) # script directory
path += '/scripts/'

os.chdir(path)
from get_graph import *

In [2]:
pwd

u'/afs/inf.ed.ac.uk/user/s13/s1338673/Desktop/STN/data'

In [3]:
file2 = open(r'clusterdata.pkl', 'rb')
clusterdata = pickle.load(file2)
file2.close()

In [4]:
file2 = open(r'mydata.pkl', 'rb')
mydata = pickle.load(file2)
file2.close()

In [5]:
file2 = open(r'deglist.pkl', 'rb')
deglist = pickle.load(file2)
file2.close()

In [6]:
deglist[:5]

[(u'guIvTBHp1oxs76q44qQ9Pg', 55.9508754, -3.1914633),
 (u'TI5mg9fDWLjt8G7Wo42U6A', 55.9508754, -3.1914633),
 (u'-Q9Npsm2YDggz5U7rvctaA', 55.9504, -3.19392),
 (u'EqRcCaLSWokuz5Z7MRpzlw', 55.9501189405058, -3.19347023963928),
 (u'ZfJcqm6k8LfsgzfgrWbLjA', 55.9499753, -3.1929207)]

In [7]:
file2 = open(r'clusters.pkl', 'rb')
clusters = pickle.load(file2)
file2.close()

In [8]:
edindata = mydata['X_Train']

In [9]:
a = clusterdata[clusterdata.index == 'mV2qIJvhYuHTI0_to25nrQ']['total'].values[0]

In [10]:
clusterdata.head()

,total,morningci,noonci,nightci,latenightci,weekci,weekendci
mV2qIJvhYuHTI0_to25nrQ,492,2703,896,46,1099,1846,4512
vxkQj8d-h_xojFOCgHBu-g,0,2,2,0,2,1,4
hwCz3dtXNGhg0TGQgSjSiA,32,117,16,1,50,73,203
hf-9dAwfK0uBuScm3pjVOQ,0,0,0,0,0,0,0
xScm9yRer2EOUU5x41yhrg,14,67,23,0,24,51,111


In [11]:
#clusterdata['mV2qIJvhYuHTI0_to25nrQ']

In [12]:
newcluster = dict()
l1 = []
for key in clusters.keys():
    l1=[]
    for node in clusters[key]:
        l1.append(node[0])
    newcluster[key[0]] = l1

In [13]:
def findcentre(row):
    for key in newcluster.keys():
        for node in newcluster[key]:
            if row['business_id'] == node:
                return key
    
    return 'NA'

In [14]:
edindata['centre'] = edindata.apply(lambda row:findcentre(row) ,axis=1)

In [15]:
def inthecol(row, col):
    c =  row['centre']
    try:
        ra = clusterdata[clusterdata.index == c][col].values[0]
        return ra
    except: 
        return 0

In [16]:
edindata['morningci'] = edindata.apply(lambda row:inthecol(row,'morningci') ,axis=1)
edindata['noonci'] = edindata.apply(lambda row:inthecol(row,'noonci') ,axis=1)
edindata['nightci'] = edindata.apply(lambda row:inthecol(row,'nightci') ,axis=1)
edindata['latenightci'] = edindata.apply(lambda row:inthecol(row,'latenightci') ,axis=1)
edindata['weekci'] = edindata.apply(lambda row:inthecol(row,'weekci') ,axis=1)
edindata['weekendci'] = edindata.apply(lambda row:inthecol(row,'weekendci') ,axis=1)

In [17]:
edindata['total'] = edindata['morningci']+edindata['noonci']+edindata['nightci']+edindata['latenightci']+edindata['weekci']+edindata['weekendci']

In [18]:
edindata['morningci'] = edindata['morningci']/ edindata['total']
edindata['noonci'] = edindata['noonci']/ edindata['total']
edindata['nightci'] = edindata['nightci']/ edindata['total']
edindata['latenightci'] = edindata['latenightci']/ edindata['total']
edindata['weekci'] = edindata['weekci']/ edindata['total']
edindata['weekendci'] = edindata['weekendci']/ edindata['total']

In [19]:
usecolumns = ['business_id' , 'attributes' , 'categories' , 'review_count', 'stars' ,'centre','morningci' , 'noonci', 'nightci','latenightci','weekci', 'weekendci']

In [20]:
usedindata = edindata[usecolumns]

In [21]:
type(usedindata)

pandas.core.frame.DataFrame

In [22]:
rows = usedindata.values.tolist()

In [23]:
import pickle

file1 = open(r'useedindata.pkl', 'wb')
pickle.dump(usedindata, file1)
file1.close()

In [24]:
lats = dict()

for name, lat, lon in deglist:
    lats[name] = (lat, lon)

In [25]:
def calcdistance(b1 , b2):

    lat1 = math.radians(float(b1[0]))
    lon1 = math.radians(float(b1[1]))
    lat2 = math.radians(float(b2[0]))
    lon2 = math.radians(float(b2[1]))
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    R = 6373.0
    a = sin(dlat/ 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [82]:
import math
import random
def get_distance(row1, row2):
    catmatch = 0
    w= 1
    w2 = 10
    w3 = 1000000
    tdist = 0
    c1 =None
    c2=None
    
    if row1[5] == 'NA' or row2[5] == 'NA':
        return 1000000
    
    catmatch = len(set(row1[2])-set(row2[2]))
    catmatchratio = (catmatch/max(len(row1[2]),len(row2[2]))) * w
    for i in range(6,12):
        dist = abs(row1[i] -row2[i])
        tdist += w2 * dist
    
    pdist = calcdistance(lats[row1[5]],lats[row2[5]]) * w3
    return catmatchratio + tdist + pdist

        #return catmatchratio + tdist + pdist 
        
    #return random.randint(0, 50)

In [72]:
from __future__ import division
def rate_instance_cluster(train_rows, instance_row, cluster_tuple, k=3):
    
    distances = []
    total = 0.0
    
    cluster_name, _, _ = cluster_tuple
    
    instance_row_cp = list(instance_row)
    instance_row_cp[5] = cluster_name
    
    for row in train_rows:
        distance = get_distance(instance_row_cp, row)
        distances.append((distance, row))
        
    distances.sort()
    #print(distances[:1])
    
    for _, row in distances[:k]:
        
        total += float(row[4])
        
    return total / k

In [73]:
def classify_instance(clusters, train_rows, instance_row, k=3):
    
    ratings = []
    counter = 0
    for cluster in clusters:
        #print('1')
        rating = rate_instance_cluster(train_rows, instance_row, cluster, k)
        ratings.append((rating, cluster))
        counter += 1
    
    ratings.sort(reverse=True)
    
    return ratings[0][1][0]
    

In [74]:
#for cluster in clusters:
#    print(cluster)

In [75]:
for d in deglist:
    if len(d) != 3:
        print(d)

In [76]:
1680 - 168

1512

In [77]:
train_rows = rows[:1512]
test_rows = rows[1512:]
row1 = rows[150]
row2 = rows[150]
print(get_distance(row1,row2))

0.0


In [83]:
m = 0
count = 0
for test_row in test_rows:
    if classify_instance(clusters.keys(), train_rows, test_row, k=5) == test_row[5]:
        m+=1
    count+=1
print(m/count)

0.0535714285714


In [ ]:
clusters.keys()